In [ ]:
# Created by: Adam Fabo
# Date: 22.5.2022
# Created at HMU Crete
# Class: Neural Networks
# File contains script to create neural network with 2 hidden layers (Chapter 6 in documentation) 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import neurolab as nl
import pandas as pd
import time
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split



In [ ]:
# load data
data = pd.read_csv('data_banknote_auth_trimmed.txt', sep=",", header=None)

# data description
# 1. variance of Wavelet Transformed image (continuous)
# 2. skewness of Wavelet Transformed image (continuous)
# 3. curtosis of Wavelet Transformed image (continuous)
# 4. entropy of image (continuous)
# 5. class (integer)

data.columns = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]

data.head()

In [ ]:
# get class as separate array
target = data.copy()["Class"]
target = target.to_numpy()

# two categories so 2 neurons, change 0 to [0,1] an 1 to [1,0]
banknotes = {0: [0,1], 1: [1,0]}
target = [ banknotes[number] for number in target]

target = np.array(target)


data = data.drop(columns = ["Class"])
data = data.to_numpy()


In [ ]:
# one hidden layer

# test size = 30%
# transfer func = LogSig
# training func = Resilient Backpropagation
# learning goal = 1e-5
# learning rate = 0.05

min_neurons = 1
max_neurons = 10

max_epochs = 3000

# for statistical errors
repeats = 10


In [ ]:
for i in range(1,max_neurons+1):
#     if i == 2:
#         a = 3
#     else:
#         a = 10
    for j in range(1,max_neurons+1):
        #round_num = 0
        #while round_num != 10:
        for round_num in range(repeats):
            print( f"Neurons {i} - {j}, Round: {round_num}" )


             # split the dataset
            X_train, X_test, y_train, y_test = train_test_split(data,target,test_size=0.3)

            # scale values to range (0,1)
            min_max_scaler = preprocessing.MinMaxScaler()
            X_train = min_max_scaler.fit_transform(X_train)
            X_test  = min_max_scaler.fit_transform(X_test)



            # create NN with i neurons in forst layers and j neurons in second layer
            net = nl.net.newff(nl.tool.minmax(X_train),[i,j,2])
            net.layers[-2].transf = nl.trans.LogSig()
            net.layers[-1].transf = nl.trans.LogSig()
            net.layers[ 0].transf = nl.trans.LogSig()


            # set train function
            net.trainf = nl.train.train_rprop

            train_time = 0


            #train for 3000 epochs
            start_time = time.time()
            error = net.train(X_train,y_train,epochs = 3000, show = 100, lr = 0.05, goal=1e-5)
            train_time += time.time() - start_time



            # test training data
            out = net.sim(X_train)
            out = np.around(out)
            correct = (out == y_train).all(axis = 1)
            accuracy_training = (np.sum(correct)/len(out))*100


            # test test data
            out = net.sim(X_test)
            out = np.around(out)
            correct = (out == y_test).all(axis = 1)
            accuracy_test = (np.sum(correct)/len(out))*100


            df = pd.DataFrame([[round_num,i,j,3000,accuracy_training,accuracy_test,train_time,error]],
                              columns = ["Round","Neurons in first layer","Neurons in second layer","Epochs","Accuracy on training","Accuracy on test","Training time","Error"])

            filename = f"2_hidden_layers_v2/{i}_{j}_neurons.csv"

            # if file exists, only append
            if os.path.isfile(filename):
                df.to_csv(filename,mode="a",header=False)
            else:
                df.to_csv(filename)




# stopped at 2-3


        
